In [1]:
from haystack_integrations.tools.mcp import MCPToolset, StreamableHttpServerInfo
from haystack_integrations.components.generators.amazon_bedrock import (
	AmazonBedrockChatGenerator,
)
from haystack.dataclasses import ChatMessage
from haystack.components.agents import Agent
from IPython.display import display, Markdown

server_info = StreamableHttpServerInfo(url="http://localhost:8000/mcp")
toolset = MCPToolset(server_info=server_info)

generator = AmazonBedrockChatGenerator(model="amazon.nova-lite-v1:0")


SYSTEM_PROMPT = """
You are a helpful assistant. 
Your task is to help users find information in the EIDC catalogue. 
The EIDC catalogue contains information about environmental science dataset. 
You have access to several tools that you should use to help answer the user's query.
You may have to use several tools in order to fulfil the user's query.
Do not use your own knowledge to answer the user's query.
Only use information that is returned from tool calls.
Your response should be structured using markdown.
All response should contain a footer with a title "References" which contains numbered links to the datasets with the information used to compose your response. 
Where the information is used should also be indicated with the number in square brackets e.g.

# Answer
This is a response to a question talking about some concept[1] found in a dataset.
# References
- [1] [Useful dataset](http://eidc.ac.uk/id/1234-5678)
"""

agent = Agent(chat_generator=generator, tools=toolset, exit_conditions=["text"])
agent.warm_up()
messages = [
	ChatMessage.from_system(SYSTEM_PROMPT),
	ChatMessage.from_user("What is the best dataset?"),
]
response = agent.run(messages=messages)


display(Markdown(response["messages"][-1].text))

# Answer
Based on the number of citations, the most cited dataset in the EIDC catalogue is "Land Cover Map 2015 (vector, GB)" with 78 citations[1]. 

# References
- [1] [Land Cover Map 2015 (vector, GB)](https://catalogue.ceh.ac.uk/id/6c6c9203-7333-4d96-88ab-78925e7a4e73)